In [0]:
import pandas as pd
import numpy as np

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [4]:
!wget  https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip

--2020-02-18 10:59:04--  https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip [following]
--2020-02-18 10:59:05--  https://www.dropbox.com/s/raw/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc18712a192c4765d64a8988a33c.dl.dropboxusercontent.com/cd/0/inline/AyXB5km_rl_21JitjgPjfGq64ewwNbAV7O4gOD0Zk4lRGlmGdVKowzOrxq_UQWNfYkqTJqqA5BaV032jO4FYG9JhIeApCGT5xBIdlZn23gKmgA/file# [following]
--2020-02-18 10:59:05--  https://uc18712a192c4765d64a8988a33c.dl.dropboxusercontent.com/cd/0/inline/AyXB5km_rl_21JitjgPjfGq64ewwNbAV7O4gOD0Zk4

In [6]:
!unzip state-farm-distracted-driver-detection.zip.1 

Archive:  state-farm-distracted-driver-detection.zip.1
  inflating: sample_submission.csv   
  inflating: driver_imgs_list.csv    
  inflating: imgs.zip                


In [7]:
!unzip imgs.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/c7/img_56699.jpg  
  inflating: train/c7/img_56717.jpg  
  inflating: train/c7/img_56724.jpg  
  inflating: train/c7/img_56768.jpg  
  inflating: train/c7/img_56779.jpg  
  inflating: train/c7/img_56849.jpg  
  inflating: train/c7/img_56899.jpg  
  inflating: train/c7/img_56938.jpg  
  inflating: train/c7/img_5706.jpg   
  inflating: train/c7/img_57076.jpg  
  inflating: train/c7/img_57122.jpg  
  inflating: train/c7/img_57137.jpg  
  inflating: train/c7/img_5715.jpg   
  inflating: train/c7/img_57278.jpg  
  inflating: train/c7/img_57321.jpg  
  inflating: train/c7/img_57345.jpg  
  inflating: train/c7/img_57355.jpg  
  inflating: train/c7/img_5741.jpg   
  inflating: train/c7/img_57433.jpg  
  inflating: train/c7/img_5753.jpg   
  inflating: train/c7/img_57559.jpg  
  inflating: train/c7/img_57594.jpg  
  inflating: train/c7/img_5771.jpg   
  inflating: train/c7/img_57712.jpg  
  inflating: train/c7/img_57733.jpg  

In [0]:
TRAIN_DIR=r'/content/train'
TEST_DIR=r'/content/test'

In [9]:
model=Sequential()
model.add(Conv2D(32, (3,3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(nb_filter=64, nb_row=3, nb_col=3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64)) # 64 neurons
model.add(Activation('relu'))
model.add(Dropout(0.4)) # drop 50% of neurons
model.add(Dense(10))
model.add(Activation('softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(3, 3), filters=64)`
  


In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [0]:
# the augmentation configuration for generating training data
train_datagen=ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

# validation image is scaled by 1/255, no other augmentation on validation data
test_datagen=ImageDataGenerator(rescale=1.0/255)

In [12]:
train_generator=train_datagen.flow_from_directory('train', target_size=(150,150), 
                                                  batch_size=32, class_mode='categorical')

Found 22424 images belonging to 10 classes.


In [16]:
validation_generator=test_datagen.flow_from_directory(r'/content/test', target_size=(150,150), 
                                                      batch_size=32, class_mode='categorical')

Found 0 images belonging to 0 classes.


In [0]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('best_model.h5',  # model filename
                             monitor='val_loss',#quantity to monitor
                        
                             verbose=0, # verbosity - 0 or 1
                             save_best_only= True, # The latest best model will not be overwritten
                             mode='auto') # The decision to overwrite model is made 
                                          # automatically depending on the quantity to monitor

In [20]:
result=model.fit_generator(train_generator, callbacks=[checkpoint],samples_per_epoch=20000, nb_epoch=25, 
                    validation_data=validation_generator, nb_val_samples=800)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., callbacks=[<keras.ca..., validation_data=<keras_pre..., steps_per_epoch=625, epochs=25, validation_steps=800)`
  


Epoch 1/25
625/625 [==============================] - 181s 289ms/step - loss: 1.7225 - acc: 0.3784
Epoch 2/25


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


625/625 [==============================] - 180s 288ms/step - loss: 0.7683 - acc: 0.7385
Epoch 3/25
625/625 [==============================] - 182s 292ms/step - loss: 0.4961 - acc: 0.8355
Epoch 4/25
625/625 [==============================] - 183s 293ms/step - loss: 0.3528 - acc: 0.8860
Epoch 5/25
625/625 [==============================] - 181s 290ms/step - loss: 0.2737 - acc: 0.9117
Epoch 6/25
625/625 [==============================] - 181s 290ms/step - loss: 0.2289 - acc: 0.9270
Epoch 7/25
625/625 [==============================] - 182s 291ms/step - loss: 0.2121 - acc: 0.9361
Epoch 8/25
625/625 [==============================] - 182s 291ms/step - loss: 0.1733 - acc: 0.9454
Epoch 9/25
625/625 [==============================] - 182s 291ms/step - loss: 0.1610 - acc: 0.9519
Epoch 10/25
625/625 [==============================] - 183s 292ms/step - loss: 0.1488 - acc: 0.9548
Epoch 11/25
625/625 [==============================] - 182s 292ms/step - loss: 0.1435 - acc: 0.9568
Epoch 12/25
625/625

In [0]:
model.save_weights('driver_state_detection.h5')

In [0]:
from tensorflow.keras.preprocessing import image

In [0]:
img = image.load_img(r'test/img_62653.jpg', target_size=(150,150))

In [0]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]

In [50]:
pred

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
train_generator.class_indices

{'c0': 0,
 'c1': 1,
 'c2': 2,
 'c3': 3,
 'c4': 4,
 'c5': 5,
 'c6': 6,
 'c7': 7,
 'c8': 8,
 'c9': 9}